In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
tqdm.pandas()
from shapely.geometry import Point
from sklearn.neighbors import NearestNeighbors
import CityHub
import glob

/tmp/ipykernel_510118/184408677.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def crossref(df, gdf):
    """
    Link rows of a dataframe to polygons in a geodataframe.
    Dataframe must have columns 'LONGITUDE' and 'LATITUDE'.
    """
    shapes = gdf.geometry
    centroids = [x.centroid for x in shapes]
    centroids = np.array([(x.x, x.y) for x in centroids])
    nn = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(centroids)
    _, indices = nn.kneighbors(df[['LONGITUDE', 'LATITUDE']])

    id_poly = []
    i = 0
    for _, row in tqdm(df.iterrows(), total=len(df)):
        point = Point(row['LONGITUDE'], row['LATITUDE'])
        new_id = None
        for idx in indices[i]:
            if shapes[idx].contains(point):
                new_id = idx
                break
        id_poly.append(new_id)
        i += 1

    df['id_poly'] = id_poly
    return df

## São Paulo

In [29]:
day_range = ["2019-12-01", "2020-01-15"]

In [30]:
hour_mapper = {
    "A NOITE" : "18:00:00",
    "A TARDE" : "15:00:00",
    "DE MADRUGADA" : "03:00:00",
    "PELA MANHÃ" : "09:00:00",
    "EM HORA INCERTA" : None,
}

In [31]:
def get_all_dates(day_range):
    dates = pd.date_range(day_range[0], day_range[1], freq="D")
    # repeat with hours: 3h, 9h, 15h, 18h
    new_dates = []
    for d in dates:
        new_dates.append(d + pd.Timedelta(hours=3))
        new_dates.append(d + pd.Timedelta(hours=9))
        new_dates.append(d + pd.Timedelta(hours=15))
        new_dates.append(d + pd.Timedelta(hours=18))
    return new_dates

In [32]:
def simplify_date(x):
    hour = x.hour
    if 6 <= hour < 12:
        new_hour = 9
    elif 12 <= hour < 18:
        new_hour = 15
    elif 18 <= hour < 24:
        new_hour = 18
    else:
        new_hour = 3
    return pd.to_datetime(f"{x.date()} {new_hour}:00:00")

In [56]:
poly_division = "SpCenterCensus2k"

### Waze

In [57]:
N = 10000000
df = pd.read_csv("data/time_series/waze-alerts.csv")
df = df[["geo", "ts", " type"]]
df = df.rename(columns = {" type" : "type"})

In [58]:
df["date"] = pd.to_datetime(df["ts"])

In [59]:
df = df[df.date >= day_range[0]]
df = df[df.date < day_range[1]]

In [60]:
df["LONGITUDE"] = df["geo"].apply(lambda x : x.split("(")[1].split(" ")[0])
df["LATITUDE"] = df["geo"].apply(lambda x : x.split(" ")[1].split(")")[0])
df.LONGITUDE = df.LONGITUDE.astype(float)
df.LATITUDE = df.LATITUDE.astype(float)
df["type"] = df["type"].apply(lambda x : "ROADCLOSED" if x == "ROAD_CLOSED" else x)
df = df[df.date >= day_range[0]]
df = df[df.date < day_range[1]]

In [61]:
gdf = gpd.read_file(f"data/shapefiles/{poly_division}.geojson")
df = crossref(df, gdf)
df = df.dropna(subset = ["id_poly"])
df["updated_date"] = df["date"].apply(simplify_date)
df["id_poly"] = df["id_poly"].astype(int)
n_poly = len(gdf)
all_dates = get_all_dates(day_range)
n_days = len(all_dates)

/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 1224134/1224134 [02:13<00:00, 9177.41it/s]


In [62]:
for t in df.type.unique():
    df_ = df[df.type == t].copy()
    df_ = df_.groupby(["id_poly", "updated_date"]).size().reset_index()
    ts = np.zeros((n_poly, n_days))
    for i, d in enumerate(all_dates):
        filtered = df_[df_.updated_date == d]
        if len(filtered) == ts.shape[0]:
            ts[:, i] = filtered[0]
        else:
            ts[filtered.id_poly, i] = filtered[0]
    
    area = gdf.to_crs("EPSG:6933").area.values / 10**6
    ts = ts / area[:, None]
    np.save(f"data/time_series/{t}_{poly_division}_Period1.npy", ts)

### Crime theft and robbery

In [63]:
df = pd.read_csv("data/time_series/furto_celular_2018_2022.csv", sep = ";")
df["hour"] = df.PERIDOOCORRENCIA.apply(lambda x : hour_mapper[x])
df["date"] = pd.to_datetime(df.DATAOCORRENCIA, format='%d/%m/%Y', errors='coerce')
df["date"] = df["date"] + pd.to_timedelta(df["hour"], errors='coerce')
print(f"Fraction of nan dates: {df.date.isna().mean():.2f}")
df = df.dropna(subset = ["date"])
df = df[["date", "LATITUDE", "LONGITUDE"]]
df = df[df.date >= day_range[0]]
df = df[df.date < day_range[1]]

/tmp/ipykernel_510118/2078175672.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/time_series/furto_celular_2018_2022.csv", sep = ";")


Fraction of nan dates: 0.41


In [64]:
gdf = gpd.read_file(f"data/shapefiles/{poly_division}.geojson")
df = crossref(df, gdf)
df = df.dropna(subset = ["id_poly"])

/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 5391/5391 [00:00<00:00, 8972.81it/s]


In [65]:
df["id_poly"] = df["id_poly"].astype(int)
n_poly = len(gdf)
all_dates = get_all_dates(day_range)
n_days = len(all_dates)
df = df.groupby(["id_poly", "date"]).size().reset_index()

In [67]:
ts = np.zeros((n_poly, n_days))
for i, d in enumerate(all_dates):
    filtered = df[df.date == d]
    if len(filtered) == ts.shape[0]:
        ts[:, i] = filtered[0]
    else:
        ts[filtered.id_poly, i] = filtered[0]

area = gdf.to_crs("EPSG:6933").area.values / 10**6
ts = ts / area[:, None]
np.save(f"data/time_series/FurtoCelular_{poly_division}_Period1.npy", ts)

### Crime robbery

In [68]:
df = pd.read_csv("data/time_series/roubo_celular_2018_2022.csv", sep = ";")
df["hour"] = df.PERIDOOCORRENCIA.apply(lambda x : hour_mapper[x])
df["date"] = pd.to_datetime(df.DATAOCORRENCIA, format='%d/%m/%Y', errors='coerce')
df["date"] = df["date"] + pd.to_timedelta(df["hour"], errors='coerce')
print(f"Fraction of nan dates: {df.date.isna().mean():.2f}")
df = df.dropna(subset = ["date"])
df = df[["date", "LATITUDE", "LONGITUDE"]]
df = df[df.date >= day_range[0]]
df = df[df.date < day_range[1]]

/tmp/ipykernel_510118/665113477.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/time_series/roubo_celular_2018_2022.csv", sep = ";")


Fraction of nan dates: 0.01


In [69]:
gdf = gpd.read_file(f"data/shapefiles/{poly_division}.geojson")
df = crossref(df, gdf)
df = df.dropna(subset = ["id_poly"])

/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 17346/17346 [00:01<00:00, 9216.71it/s]


In [70]:
df["id_poly"] = df["id_poly"].astype(int)
n_poly = len(gdf)
all_dates = get_all_dates(day_range)
n_days = len(all_dates)
df = df.groupby(["id_poly", "date"]).size().reset_index()

In [71]:
ts = np.zeros((n_poly, n_days))
for i, d in enumerate(all_dates):
    filtered = df[df.date == d]
    if len(filtered) == ts.shape[0]:
        ts[:, i] = filtered[0]
    else:
        ts[filtered.id_poly, i] = filtered[0]

area = gdf.to_crs("EPSG:6933").area.values / 10**6
ts = ts / area[:, None]
np.save(f"data/time_series/RouboCelular_{poly_division}_Period1.npy", ts)

### Unify time-series

In [72]:
features_naming = {
    "JAM": "Jam",
    "ACCIDENT": "Accident",
    "ROADCLOSED": "Road Closed",
    "WEATHERHAZARD": "Weather Hazard",
    "FurtoCelular": "Phone Theft",
    "RouboCelular": "Phone Robbery"
}

In [73]:
ts = [np.load(f"data/time_series/{key}_{poly_division}_Period1.npy") for key in features_naming.keys()]
ts_sum = [np.sum(t) for t in ts]
ts = [t for i, t in enumerate(ts) if ts_sum[i] > 0]
signals = [s for i, s in enumerate(features_naming.keys()) if ts_sum[i] > 0]

In [74]:
date = get_all_dates(day_range)
df = []
for poly in range(ts[0].shape[0]):
    for t in range(ts[0].shape[1]):
        df.append({
            "date" : date[t],
            "id_poly" : poly,
        })
        for i, signal in enumerate(signals):
            df[-1][
                features_naming[signal]
            ] = ts[i][poly, t]

df = pd.DataFrame(df)
print(df.shape)
df.to_csv(f"data/polygon_data/{poly_division}_Period1.csv", index=False)

(368000, 8)


## NY

In [3]:
df = []
for i in range(1, 7):
    df_ = pd.read_csv(f"data/time_series/yellow_taxi_2016_0{i}.csv")
    # remove rows with trip_distance bigger than 500km
    df_ = df_[df_.trip_distance < 500]
    df_ = df_.sample(10**6//6, random_state = 0)
    df.append(df_)
df = pd.concat(df)

In [4]:
gdf = gpd.read_file("data/shapefiles/NYBlocks.geojson")

In [5]:
df["LONGITUDE"] = df["pickup_longitude"]
df["LATITUDE"] = df["pickup_latitude"]
df["date"] = pd.to_datetime(df["tpep_pickup_datetime"])
df = df.drop(columns = ["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "tpep_dropoff_datetime", "tpep_pickup_datetime"], axis = 1)

/tmp/ipykernel_369222/2284987300.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["tpep_pickup_datetime"])


In [6]:
df = crossref(df, gdf)
df = df.dropna(subset = ["id_poly"])

/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 999996/999996 [01:36<00:00, 10312.96it/s]


In [7]:
day_ns = 60*60*24*10**9
df["date_int"] = df.date.astype(int)
df["date_int"] = df.date_int / day_ns
df["date_int"] = df.date_int.astype(int)    
df = df.groupby(["id_poly", "date_int"]).agg({"passenger_count" : "sum", "trip_distance" : "mean", "date" : ["min", "count"]}).reset_index()
df.columns = ["id_poly", "date_int", "passanger_count", "trip_distance", "date", "n_pickups"]
df["id_poly"] = df["id_poly"].astype(int)

In [8]:
# remove hour from date
df["date"] = pd.to_datetime(df["date"]).dt.floor('d')

In [9]:
ny_range = ["2016-01-01", "2016-07-01"]
day_ns = 60*60*24*10**9
time_aux = pd.Series(ny_range).apply(pd.to_datetime)
time_aux = (time_aux.astype(int) / day_ns).astype(int)
min_date = time_aux.min()
max_date = time_aux.max()
n_days = int(max_date - min_date)
n_poly = len(gdf)
ts = np.zeros((n_poly, n_days, 3))
k = 0
for day in range(min_date, max_date):
    filtered = df[df.date_int == day]
    if len(filtered) == ts.shape[0]:
        ts[:, k, :] = filtered[["passanger_count", "trip_distance", "n_pickups"]]
    else:
    # fill missing values with 0
        ts[:, k] = 0
        ts[filtered.id_poly, k, :] = filtered[["passanger_count", "trip_distance", "n_pickups"]]
    k += 1

# change to a projection with meters as unit
area = gdf.to_crs("EPSG:6933").area.values / 10**6
ts = ts / area[:, None, None]

In [10]:
features = ["PassangerCount", "TripDistance", "NPickups"]
for i in range(3):
    np.save(f"data/time_series/{features[i]}_NYBlocks_Day.npy", ts[:, :, i])

#### dropout

In [11]:
df = []
for i in range(1, 7):
    df_ = pd.read_csv(f"data/time_series/yellow_taxi_2016_0{i}.csv")
    df_ = df_[df_.trip_distance < 500]
    df_ = df_.sample(10**6//6, random_state = 0)
    
    df.append(df_)
df = pd.concat(df)

In [12]:
df["LONGITUDE"] = df["dropoff_longitude"]
df["LATITUDE"] = df["dropoff_latitude"]
df["date"] = pd.to_datetime(df["tpep_dropoff_datetime"])
df = df[["LONGITUDE", "LATITUDE", "date"]]

/tmp/ipykernel_369222/959207515.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["tpep_dropoff_datetime"])


In [13]:
gdf = gpd.read_file("data/shapefiles/NYBlocks.geojson")

In [14]:
df = crossref(df, gdf)
df = df.dropna(subset = ["id_poly"])

/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
100%|██████████| 999996/999996 [01:35<00:00, 10432.41it/s]


In [15]:
day_ns = 60*60*24*10**9
df["date_int"] = df.date.astype(int)
df["date_int"] = df.date_int / day_ns
df["date_int"] = df.date_int.astype(int)    
df = df.groupby(["id_poly", "date_int"]).size().reset_index()
df.columns = ["id_poly", "date_int", "count"]
df["id_poly"] = df["id_poly"].astype(int)

In [16]:
ny_range = ["2016-01-01", "2016-07-01"]
day_ns = 60*60*24*10**9
time_aux = pd.Series(ny_range).apply(pd.to_datetime)
time_aux = (time_aux.astype(int) / day_ns).astype(int)
min_date = time_aux.min()
max_date = time_aux.max()
n_days = int(max_date - min_date)
n_poly = len(gdf)
ts = np.zeros((n_poly, n_days))
k = 0
for day in range(min_date, max_date):
    filtered = df[df.date_int == day]
    if len(filtered) == ts.shape[0]:
        ts[:, k] = filtered["count"]
    else:
    # fill missing values with 0
        ts[:, k] = 0
        ts[filtered.id_poly, k] = filtered["count"]
    k += 1

# change to a projection with meters as unit
area = gdf.to_crs("EPSG:6933").area.values / 10**6
ts = ts / area[:, None]

In [17]:
np.save(f"data/time_series/NDropoffs_NYBlocks_Day.npy", ts)

### Unify time-series

In [18]:
signals = glob.glob("data/time_series/*NYBlocks*")
signals = [s.split("_")[-3] for s in signals]
signals = [s.split("/")[-1] for s in signals]
signals

['NDropoffs', 'NPickups', 'PassangerCount', 'TripDistance']

In [19]:
poly_division = "NYBlocks"
time_interval = "Day"
ts = [
    np.load(f"data/time_series/{signal}_{poly_division}_{time_interval}.npy") for signal in signals
]
# drop if all values are 0
ts_sum = [np.sum(t) for t in ts]
ts = [t for i, t in enumerate(ts) if ts_sum[i] > 0]
signals = [s for i, s in enumerate(signals) if ts_sum[i] > 0]
df = []
date = pd.date_range(start = ny_range[0], end = ny_range[1], freq = "d")
poly_id = np.arange(ts[0].shape[0])
for poly in range(ts[0].shape[0]):
    for t in range(ts[0].shape[1]):
        df.append({
            "date" : date[t],
            "id_poly" : poly_id[poly],
        })
        for i, signal in enumerate(signals):
            df[-1][signal] = ts[i][poly, t]

df = pd.DataFrame(df)
print(df.shape)
df.to_csv(f"data/polygon_data/{poly_division}_{time_interval}.csv", index=False)

(212576, 6)


## BLA Cities

In [75]:
gdf = gpd.read_file("data/shapefiles/BLACities.geojson")
name = gdf.name.tolist()
id_poly = gdf.id_poly.tolist()
df = pd.read_csv("data/time_series/cities_time_series.csv")

In [77]:
father_classes_hierarchy = {
    "floresta": [
        "formacao_florestal",
        "formacao_savanica",
        "mangue",
        "restinga_arborizada",
    ],
    "formacao_natural_nao_florestal": [
        "campo_alagado_e_area_pantanosa",
        "formacao_campestre",
        "apicum",
        "afloramento_rochoso",
        "outras_formacoes_nao_florestais",
    ],
    "agropecuaria": [
        "pastagem",
        "soja",
        "cana",
        "arroz",
        "outras_lavouras_temporarias",
        "cafe",
        "citrus",
        "outras_lavouras_perenes",
        "floresta_plantada",
        "moisaco_de_agricultura_e_pastagem",
    ],
    "area_nao_vegetada": [
        "praia_e_duna",
        "infraestrutura_urbana",
        "mineracao",
        "outras_areas_nao_vegetadas",
    ],
    "corpo_dagua": ["rio_lago_e_oceano", "aquicultura"],
    "nao_observado": [
        "nao_observado",  # "nao_classificado"
    ],
}


In [78]:
for father, children in father_classes_hierarchy.items():
    df[father] = df[children].sum(axis = 1)

In [84]:
selected_columns = ["formacao_florestal", "formacao_savanica", "formacao_natural_nao_florestal", "pastagem", "soja", "cana", "mineracao", "infraestrutura_urbana", "moisaco_de_agricultura_e_pastagem", "mangue"]
features_naming = {
    "formacao_florestal" : "Forest Formation",
    "formacao_savanica" : "Cerrado",
    "formacao_natural_nao_florestal" : "Natural Formation",
    "pastagem" : "Pasture",
    "soja" : "Soy",
    "cana" : "Sugar Cane",
    "mineracao" : "Mining",
    "infraestrutura_urbana" : "Urban Infrastructure",
    "moisaco_de_agricultura_e_pastagem" : "Mosaic Agriculture",
    "mangue" : "Mangrove"
}

In [85]:
df_result = []
for year in df.year.unique():
    df_ = df[df.year == year]
    new_df = {
        "id_poly" : id_poly,
    }
    for column in selected_columns:
        new_df[features_naming[column]] = df_[column]
    new_df = pd.DataFrame(new_df)
    new_df["date"] = pd.to_datetime(f"{year}-01-01")
    df_result.append(new_df)
df_result = pd.concat(df_result)

In [86]:
#place date and id_poly as first columns
cols = df_result.columns.tolist()
cols.remove("date")
cols.remove("id_poly")
cols = ["date", "id_poly"] + cols
df_result = df_result[cols]
df_result = df_result.sort_values(["id_poly", "date"])
df_result.to_csv("data/polygon_data/BLACities_Year.csv", index=False)